| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

This notebook is licensed under GPL 3.0. Please visit our [Github repo](https://github.com/edgi-govdata-archiving/ECHO-Cross-Program) for more information.

The notebook was collaboratively authored by EDGI following our [authorship protocol](https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/).

For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run
* A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# **Let's begin!**

Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

These first two cells give us access to some external Python code we will need.

### 1.  Bring in some code that is stored in a Github project, and run a few Python modules.

In [ ]:
# I installed dependencies in a python virtual env for this exercise
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
!pip install geopandas &>/dev/null;

import warnings
warnings.filterwarnings('ignore')

# Added for new section 5.a
from getpass import getpass
import requests
import xmltodict

print("Done!")

### 2.  Run this next cell to select which geography you wish to view ECHO data by.
Choose the type of region and proceed to the next cell.

In [ ]:
from ECHO_modules.utilities import show_region_type_widget

region_type_widget = show_region_type_widget()
state_widget = None

### 3.  Select your state of interest.  _If you are interested in a zip code analysis, this can be skipped._

In [ ]:
from ECHO_modules.utilities import show_state_widget

state_widget = show_state_widget()

### 4.  Run this cell and choose the specific region you want to look at.
It will ask you to specify which place you want to look at. If you want to look at an entire state, you should still run this cell, but you won't see any output.

For reference, here is a map of congressional districts: https://www.govtrack.us/congress/members/map

In [ ]:
from ECHO_modules.utilities import show_pick_region_widget

region_widget = None
region_type = region_type_widget.value
if ( region_type != 'State' ):
    region_widget = show_pick_region_widget( type=region_type,
                                           state_widget=state_widget )

### 5. Chart the top violators in the region for CAA, CWA and RCRA.
The charts show the 20 facilities with the most quarters in non-compliance with these environmental protection laws over the past 3 years.

In [ ]:
# Helper function for section 5.a to collect top 20 violators in each section
def violator_facs(flag, df):
    return [{"flag": flag, "FAC_NAME": i} for i in list(df.FAC_NAME)]

In [ ]:
from ECHO_modules.utilities import get_active_facilities, get_top_violators, chart_top_violators

state = state_widget.value if state_widget is not None else None
region_selected = None

violators = list()

if ( region_type != 'State' ):
    region_selected = region_widget.value
df_active = get_active_facilities( state, region_type, region_selected )

df_violators = get_top_violators( df_active, 'AIR_FLAG',  
        'CAA_3YR_COMPL_QTRS_HISTORY', 'CAA_FORMAL_ACTION_COUNT', 20 )
violators.extend(violator_facs("AIR_FLAG", df_violators))
display( chart_top_violators( df_violators, state, region_selected, 'CAA' ))

df_violators = get_top_violators( df_active, 'NPDES_FLAG', 
        'CWA_13QTRS_COMPL_HISTORY', 'CWA_FORMAL_ACTION_COUNT', 20 )
violators.extend(violator_facs("AIR_FLAG", df_violators))
display( chart_top_violators( df_violators, state, region_selected, 'CWA' ))

df_violators = get_top_violators( df_active, 'RCRA_FLAG',  
        'RCRA_3YR_COMPL_QTRS_HISTORY', 'RCRA_FORMAL_ACTION_COUNT', 20 )
violators.extend(violator_facs("AIR_FLAG", df_violators))
display( chart_top_violators( df_violators, state, region_selected, 'RCRA' ))


#### 5.a Exploration of methods to tie in campaign contributions
This section looks at a process to bring in information from opensecrets.org on campaign contributions based on the active "violators" and a selection of a particular Congressional District. The idea came from the exercise that Kelsey ran us through in the ESIP Summer Meeting (July 23, 2021). This is a work in progress exploration of the idea with some thoughts on how it could be put into production.

To facilitate this, I grabbed an API key from opensecrets.org to see what that looks like. My conclusion is that it might be useful to return and work with closer to real time information for those things they are tracking that change often. However, to actually use the dataset, establishing connections between organization/facility identifiers and other properties would be better done with some prep work using bulk data from OpenSecrets. I requested that as well, but my request needs to be approved.

The following codeblocks are not something you'd want to include in a notebook like this. I provide it here to share some of the thinking and what might need to be done if you wanted to run a backend process via API vs. bulk data access.

In [ ]:
# Get API key into memory with getpass()
opensecrets_api_key = getpass("OPEN SECRETS API KEY: ")

I put together a couple of things as functions, mostly to handle converting XML responses from the OpenSecrets API into python objects, which I find much easier to work with.

In [ ]:
def os_check_org(org_name):
    r = requests.get(f"https://www.opensecrets.org/api/?method=getOrgs&org={org_name}&apikey={opensecrets_api_key}")
    if r.status_code != 200:
        return
    
    d_org_response = xmltodict.parse(r.text, dict_constructor=dict)

    if "response" in d_org_response and "organization" in d_org_response["response"]:
        if isinstance(d_org_response["response"]["organization"], dict):
            org_list = [d_org_response["response"]["organization"]]
        else:
            org_list = d_org_response["response"]["organization"]

        orgs = list()
        for org in org_list:
            new_org = dict()
            for k,v in org.items():
                new_org[k.replace("@","")] = v
                orgs.append(new_org)
            
        return orgs

def os_legislators(state):
    r = requests.get(f"http://www.opensecrets.org/api/?method=getLegislators&id={state}&apikey={opensecrets_api_key}")
    if r.status_code != 200:
        return
    
    d_legislators = xmltodict.parse(r.text, dict_constructor=dict)
    return d_legislators
    
def os_org_contributions(orgid):
    r = requests.get(f"http://www.opensecrets.org/api/?method=orgSummary&id={orgid}&apikey={opensecrets_api_key}")
    if r.status_code != 200:
        return
    
    d_org_contribs = xmltodict.parse(r.text, dict_constructor=dict)
    return d_org_contribs

def os_candidate(state, district):
    state_district = f"{state}0{str(district)}"
    legislators = os_legislators(state)
    candidate = next((i for i in legislators["response"]["legislator"] if i["@office"] == state_district), None)
    if candidate is not None:
        return candidate
    else:
        return

def os_cand_contributions(cid, cycle="2020"):
    r = requests.get(f"https://www.opensecrets.org/api/?method=candContrib&cid={cid}&cycle={cycle}&apikey={opensecrets_api_key}")
    if r.status_code != 200:
        return
    
    d_cand_contribs = xmltodict.parse(r.text, dict_constructor=dict)
    return d_cand_contribs


Notionally, in an app liks this we would want something responsive to the user input, diving into a particular state, county, congressional district, etc. I added something to the previous workflow to build top violators and plot those to add each set to an array for further use. That gives us the top 20 violators and the category/flag. I then pull out the unique "FAC_NAME" values, which appear to be the main thing to try and match to organization names in OpenSecrets.

In [ ]:
print(region_type_widget.value)
print(state_widget.value)
print(region_widget.value)

violator_unique_names = list(set([i["FAC_NAME"] for i in violators]))
print(len(violator_unique_names))

This is where the real bottlenecks are going to be and why a bulk data process to line up OpenSecrets data with what EDGI has in their data system will really be the way to go. There are thousands of individual facility names in the underlying EPA data, and trying to run those through a lookup process one at a time based on the OpenSecrets REST API is really not feasible. You might also need to pull mutiple pieces of information on facilities and organizations in the EPA data together in order to establish more certain links to OpenSecrets data. You could probably run these queries in parallel, but that might piss someone off on the OpenSecrets end.

In [ ]:
%%time
os_fac_matches = list()
for name in violator_unique_names:
    os_fac_matches.append({
        "edgi_fac_name": name,
        "os_org_records": os_check_org(name)
    })


From the looped check through our top violators for LA Congressional District 1, we get just a few hits. We also notice that the flightly fuzzy search on the OpenSecrets end come up with multiple results that might introduce some uncertainty.

In [ ]:
violators

Tying things back together from unique names and cleaning up the results for use, I only consider cases where there's an exact match on name and add unique OpenSecrets orgid values to the basic information collected as our top violators in each of three categories.

In [ ]:
top_violator_os_matches = list()
for item in [i for i in os_fac_matches if i["os_org_records"] is not None]:
    possible_matches = [i for i in item["os_org_records"] if i["orgname"].lower() == item["edgi_fac_name"].lower()]
    if possible_matches:
        edgi_record = next((i for i in violators if i["FAC_NAME"] == item["edgi_fac_name"]), None)
        if edgi_record is not None:
            edgi_record["os_orgids"] = list(set([i["orgid"] for i in possible_matches]))
            top_violator_os_matches.append(edgi_record)


Out of the top violators, we only have one that has an exact match by name. That's a starting point, but we really need to do the harder work of munging things together on who these organizations actually are and getting as many identifiers lined up between the two systems as we can. Then everything else will come together.

In [ ]:
top_violator_os_matches

This is where I've kind of begged off this particular approach. We can hit another API end point from OpenSecrets on orgids we've managed to link and get campaign contrbutions, but what we're seeing is highly denormalized data where we have lots of other identifiers we have to put back together. If you go visit the web site referenced in this record, we can see that "BAY LTD" gave $367 from an individual to the campaign of Steve Scalise in 2020.

In [ ]:
for v in top_violator_os_matches:
    print(v["FAC_NAME"], v["flag"])
    for orgid in v["os_orgids"]:
        display(os_org_contributions(orgid))

I took this about as far as I can from this point. The actual funding data through the API appears to be summarized to the top (n) contributions and will not provide everything. I can plug in state and district to get the candidate and their ID and then get their top contributions for a given election cycle, but this does not turn anything up for our particular violator that we could get a match on. The web page lists a number for Bay Ltd, so it must be in OpenSecrets' data. We'll have to see if they supply that in their bulk data download.

In [ ]:
candidate = os_candidate(state_widget.value, region_widget.value)
candidate

In [ ]:
contributions = os_cand_contributions(candidate["@cid"])
contributions